In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator, ClassifierMixin
from imblearn.over_sampling import SMOTE

In [7]:
# Define the KNN class
class KNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, distance_metric='euclidean'):
        self.k = k
        self.distance_metric = distance_metric

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        distances = [self.compute_distance(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train.iloc[i] for i in k_indices]
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

    def compute_distance(self, X1, X2):
        if self.distance_metric == 'euclidean':
            return np.sqrt(np.sum((X1 - X2) ** 2))
        elif self.distance_metric == 'manhattan':
            return np.sum(np.abs(X1 - X2))
        else:
            raise ValueError("Unsupported distance metric")

In [8]:
# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Handle categorical variables
    combined_data = pd.concat([train_data, test_data], axis=0)
    label_encoders = {}
    for column in combined_data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        combined_data[column] = le.fit_transform(combined_data[column])
        label_encoders[column] = le

    train_data = combined_data.iloc[:len(train_data)]
    test_data = combined_data.iloc[len(train_data):]

    # Separate features and target
    X_train = train_data.drop(columns=['Exited'])
    y_train = train_data['Exited']
    X_test = test_data.drop(columns=['Exited'])

    # Feature engineering: Polynomial features
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Scale features
    scaler = StandardScaler()
    X_train_poly = scaler.fit_transform(X_train_poly)
    X_test_poly = scaler.transform(X_test_poly)

    return X_train_poly, y_train, X_test_poly

In [9]:
# Define cross-validation function
def cross_validate(X, y, knn, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        knn.fit(X_train, y_train)
        y_pred = knn.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return scores

In [10]:

# Load and preprocess data
X, y, X_test = preprocess_data('cs-506-predicting-customer-churn-using-knn/train.csv', 'cs-506-predicting-customer-churn-using-knn/test.csv')

# Balance the dataset with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Hyperparameter tuning with GridSearchCV
param_grid = {
    'k': [3, 5, 7, 9, 11], 
    'distance_metric': ['euclidean', 'manhattan']
}
grid_search = GridSearchCV(KNN(), param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_resampled, y_resampled)

# Display best parameters and best cross-validation score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

# Train with optimal hyperparameters on the full balanced dataset
best_knn = KNN(k=grid_search.best_params_['k'], distance_metric=grid_search.best_params_['distance_metric'])
best_knn.fit(X_resampled, y_resampled)
test_predictions = best_knn.predict(X_test)

# Save test predictions to CSV file
submission = pd.DataFrame({'id': pd.read_csv('cs-506-predicting-customer-churn-using-knn/test.csv')['id'], 'Exited': test_predictions})
submission.to_csv('submissions_optimized.csv', index=False)

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:993: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 982, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 253, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 344, in _score
    response_method = _check_response_method(estimator, self._response_method)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Best parameters: {'distance_metric': 'euclidean', 'k': 3}
Best cross-validation score: nan


In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator, ClassifierMixin
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Define data preprocessing function
def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    # Handle categorical variables
    combined_data = pd.concat([train_data, test_data], axis=0)
    label_encoders = {}
    for column in combined_data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        combined_data[column] = le.fit_transform(combined_data[column])
        label_encoders[column] = le

    train_data = combined_data.iloc[:len(train_data)]
    test_data = combined_data.iloc[len(train_data):]

    # Separate features and target
    X_train = train_data.drop(columns=['Exited'])
    y_train = train_data['Exited']
    X_test = test_data.drop(columns=['Exited'])

    # Feature engineering: Polynomial features
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Scale features
    scaler = StandardScaler()
    X_train_poly = scaler.fit_transform(X_train_poly)
    X_test_poly = scaler.transform(X_test_poly)

    return X_train_poly, y_train, X_test_poly

# Define cross-validation function
def cross_validate(X, y, model, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return scores

# Load and preprocess data
X, y, X_test = preprocess_data('cs-506-predicting-customer-churn-using-knn/train.csv', 'cs-506-predicting-customer-churn-using-knn/test.csv')

# Balance the dataset with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Hyperparameter tuning with RandomizedSearchCV for RandomForest
param_dist_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
random_search_rf = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_dist_rf, n_iter=10, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42)
random_search_rf.fit(X_resampled, y_resampled)

# Display best parameters and best cross-validation score for RandomForest
print("Best parameters for RandomForest:", random_search_rf.best_params_)
print("Best cross-validation score for RandomForest:", random_search_rf.best_score_)

# Train with optimal hyperparameters on the full balanced dataset using RandomForest
best_rf = random_search_rf.best_estimator_
best_rf.fit(X_resampled, y_resampled)
test_predictions_rf = best_rf.predict(X_test)

# Save test predictions to CSV file for RandomForest
submission_rf = pd.DataFrame({'id': pd.read_csv('cs-506-predicting-customer-churn-using-knn/test.csv')['id'], 'Exited': test_predictions_rf})
submission_rf.to_csv('submissions_rf.csv', index=False)

# Hyperparameter tuning with RandomizedSearchCV for XGBoost
param_dist_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}
random_search_xgb = RandomizedSearchCV(XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'), param_dist_xgb, n_iter=10, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42)
random_search_xgb.fit(X_resampled, y_resampled)

# Display best parameters and best cross-validation score for XGBoost
print("Best parameters for XGBoost:", random_search_xgb.best_params_)
print("Best cross-validation score for XGBoost:", random_search_xgb.best_score_)

# Train with optimal hyperparameters on the full balanced dataset using XGBoost
best_xgb = random_search_xgb.best_estimator_
best_xgb.fit(X_resampled, y_resampled)
test_predictions_xgb = best_xgb.predict(X_test)

# Save test predictions to CSV file for XGBoost
submission_xgb = pd.DataFrame({'id': pd.read_csv('cs-506-predicting-customer-churn-using-knn/test.csv')['id'], 'Exited': test_predictions_xgb})
submission_xgb.to_csv('submissions_xgb.csv', index=False)

Best parameters for RandomForest: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 20}
Best cross-validation score for RandomForest: 0.8337426985185419


/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:19:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:19:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:19:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:19:24] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/x

Best parameters for XGBoost: {'subsample': 0.8, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.01, 'colsample_bytree': 0.8}
Best cross-validation score for XGBoost: 0.821290827324596


/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:19:47] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [15]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.model_selection import KFold, RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier

def preprocess_data(train_path, test_path):
    train_data = pd.read_csv(train_path)
    test_data = pd.read_csv(test_path)

    combined_data = pd.concat([train_data, test_data], axis=0)
    label_encoders = {}
    for column in combined_data.select_dtypes(include=['object']).columns:
        le = LabelEncoder()
        combined_data[column] = le.fit_transform(combined_data[column])
        label_encoders[column] = le

    train_data = combined_data.iloc[:len(train_data)]
    test_data = combined_data.iloc[len(train_data):]

    X_train = train_data.drop(columns=['Exited'])
    y_train = train_data['Exited']
    X_test = test_data.drop(columns=['Exited'])

    # Feature engineering
    poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
    X_train_poly = poly.fit_transform(X_train)
    X_test_poly = poly.transform(X_test)

    # Feature scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_poly)
    X_test_scaled = scaler.transform(X_test_poly)

    return X_train_scaled, y_train, X_test_scaled

def cross_validate(X, y, model, n_splits=5):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in skf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_val)[:, 1]
        score = roc_auc_score(y_val, y_pred)
        scores.append(score)

    return np.mean(scores)

# Load and preprocess data
X, y, X_test = preprocess_data('cs-506-predicting-customer-churn-using-knn/train.csv', 'cs-506-predicting-customer-churn-using-knn/test.csv')

# Balance the dataset with SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Feature selection
selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=42), max_features=20)
X_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

# Define models
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
lgb = LGBMClassifier(random_state=42)
gb = GradientBoostingClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
svm = SVC(probability=True, random_state=42)
mlp = MLPClassifier(random_state=42)

# Define hyperparameter search spaces
param_dist = {
    'rf': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'xgb': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    },
    'lgb': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'num_leaves': [31, 63, 127]
    },
    'gb': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0]
    },
    'lr': {
        'C': [0.1, 1, 10],
        'penalty': ['l1', 'l2']
    },
    'svm': {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'mlp': {
        'hidden_layer_sizes': [(50,), (100,), (50, 50)],
        'alpha': [0.0001, 0.001, 0.01],
        'learning_rate': ['constant', 'adaptive']
    }
}

# Perform hyperparameter tuning
best_models = {}
for name, model in [('rf', rf), ('xgb', xgb), ('lgb', lgb), ('gb', gb), ('lr', lr), ('svm', svm), ('mlp', mlp)]:
    print(f"Tuning {name}...")
    random_search = RandomizedSearchCV(model, param_dist[name], n_iter=10, cv=5, scoring='roc_auc', n_jobs=-1, random_state=42)
    random_search.fit(X_selected, y_resampled)
    best_models[name] = random_search.best_estimator_
    print(f"Best parameters for {name}: {random_search.best_params_}")
    print(f"Best cross-validation score for {name}: {random_search.best_score_}")

# Create ensemble model
ensemble = VotingClassifier(
    estimators=[(name, model) for name, model in best_models.items()],
    voting='soft'
)

# Train ensemble model
ensemble.fit(X_selected, y_resampled)

# Make predictions
ensemble_predictions = ensemble.predict_proba(X_test_selected)[:, 1]

# Save predictions to CSV file
submission = pd.DataFrame({'id': pd.read_csv('cs-506-predicting-customer-churn-using-knn/test.csv')['id'], 'Exited': ensemble_predictions})
submission.to_csv('ensemble_submissions.csv', index=False)

print("Ensemble model trained and predictions saved.")

Tuning rf...
Best parameters for rf: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': 30}
Best cross-validation score for rf: 0.971913041287323
Tuning xgb...


/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:27:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:27:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:27:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:27:41] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/homebrew/lib/python3.11/site-packages/x

Best parameters for xgb: {'subsample': 1.0, 'n_estimators': 200, 'max_depth': 6, 'learning_rate': 0.2, 'colsample_bytree': 0.8}
Best cross-validation score for xgb: 0.9713621509892827
Tuning lgb...
[LightGBM] [Info] Number of positive: 9574, number of negative: 9573
[LightGBM] [Info] Number of positive: 9573, number of negative: 9574
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004096 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3541
[LightGBM] [Info] Number of data points in the train set: 19147, number of used features: 14
[LightGBM] [Info] Number of positive: 9573, number of negative: 9574
[LightGBM] [Info] Number of positive: 9574, number of negative: 9573
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500026 -> initscore=0.000104
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006610

/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_search.py:318: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py", line

Best parameters for lr: {'penalty': 'l2', 'C': 10}
Best cross-validation score for lr: 0.8852660815778908
Tuning svm...
Best parameters for svm: {'kernel': 'poly', 'gamma': 'auto', 'C': 10}
Best cross-validation score for svm: 0.9305294105529148
Tuning mlp...


...

Best parameters for mlp: {'learning_rate': 'adaptive', 'hidden_layer_sizes': (50, 50), 'alpha': 0.001}
Best cross-validation score for mlp: 0.9449706398658616


/opt/homebrew/lib/python3.11/site-packages/xgboost/core.py:158: UserWarning: [23:35:50] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 11967, number of negative: 11967
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000427 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3537
[LightGBM] [Info] Number of data points in the train set: 23934, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with pos

/opt/homebrew/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Ensemble model trained and predictions saved.
